In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
from sklearn.metrics import  classification_report

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-28 00:08:29.489644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 00:08:29.493193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 00:08:29.493326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
#Load in data
label_list = ['backward','forward','stop','left','right']
img_dir = []
labels = []

def get_path_list(label):
    print(label)
    dir_path = 'data/{}'.format(label)
    data = []
    labels = []
    for file in os.listdir(dir_path):
        data.append(os.path.join(dir_path,file))
        labels.append(label)
    return data,labels

def get_all_paths(label_list):
    for label in label_list:
        temp_data, temp_labels = get_path_list(label)
        img_dir.extend(temp_data)
        labels.extend(temp_labels)
get_all_paths(label_list)

backward
forward
stop
left
right


In [3]:
len(img_dir),len(labels)

(500, 500)

In [4]:
unsplit_data = {'Image':img_dir,'Label':labels}
unsplit_data = pd.DataFrame(unsplit_data)

In [5]:
for i in unsplit_data['Image']:
    print(i)

data/backward/06ca0e75-b6b4-4b97-9afe-a727b0143ea4.jpg
data/backward/02e528ad-59fc-4757-9121-ffbfa44c485f.jpg
data/backward/042fa723-00b9-463a-a91b-719e8c5a53c2.jpg
data/backward/05703d63-4bd3-49b3-a236-e06d7dde5e58.jpg
data/backward/04e82298-d2e3-4944-a6d5-67c4eefc4a80.jpg
data/backward/00af742c-20c2-4f9e-a31a-4503ea86a8bb.jpg
data/backward/02638b22-b4a8-44d3-b726-ad9858d1f1f5.jpg
data/backward/066680a3-ffca-490e-943c-7938580b05f2.jpg
data/backward/02dcafaa-4eec-4052-b014-a1d766fc47a5.jpg
data/backward/039cb58d-d9bf-43a8-9cd0-a0fe460202c3.jpg
data/backward/02db8bf6-3fd5-4d69-9e8d-640bc105b44c.jpg
data/backward/00aa11ca-ec30-48f0-91da-bb717bdf8b57.jpg
data/backward/00c77059-12b4-4334-93e5-545d100179fb.jpg
data/backward/02887c7f-4bac-411b-aeae-172b2e3d01ef.jpg
data/backward/00c6bd08-0c39-44d9-8d90-bc89df46316a.jpg
data/backward/0598ef3e-2ea1-449a-9735-f4aa82d5ff2c.jpg
data/backward/03a5f33c-28a9-440b-b184-81bda60999d6.jpg
data/backward/04ea9e7b-7596-422b-b965-83db0b2c8059.jpg
data/backw

In [6]:
unsplit_data['Label'].value_counts()

backward    100
forward     100
stop        100
left        100
right       100
Name: Label, dtype: int64

In [7]:
#Split data into 80,10,10

from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(unsplit_data['Image'],unsplit_data['Label'],test_size=.2,random_state=42)

In [8]:
unsplit_data = {'Image':x_val,'Label':y_val}
unsplit_data = pd.DataFrame(unsplit_data)
x_val,x_test,y_val,y_test = train_test_split(unsplit_data['Image'],unsplit_data['Label'],test_size=.5,random_state=42)

train_gen = pd.DataFrame({'Image':x_train,'Label':y_train})
val_gen = pd.DataFrame({'Image':x_val,'Label':y_val})
test_gen = pd.DataFrame({'Image':x_test,'Label':y_test})
val_gen = pd.DataFrame({'Image':x_val,'Label':y_val})


In [9]:
train_gen

,Image,Label
249,data/stop/017e6c4c-2945-485f-9c1b-1d39f00c524e...,stop
433,data/right/062b0c5d-e05b-4b2e-9aab-2f098bb53cd...,right
19,data/backward/002e5485-87dd-4b79-a9ef-c6a1d723...,backward
322,data/left/03ac5380-8674-41da-844b-dc1946f04e3e...,left
332,data/left/00fecf4f-1823-4e02-ab34-c9fa6c6a79f5...,left
...,...,...
106,data/forward/00bea9e7-add1-4cc3-b4bc-bcf234966...,forward
270,data/stop/01340e0c-d05f-4459-b023-8afa3f3d5110...,stop
348,data/left/031d227a-7c95-499d-9e55-ade28d7e6f57...,left
435,data/right/0254066e-45c7-4f2c-a33f-c33f24f2d71...,right


In [10]:
val_gen

,Image,Label
76,data/backward/05037a97-c299-434a-b31a-06ca099a...,backward
155,data/forward/020cd45b-a632-4bd1-b769-c66c8806f...,forward
30,data/backward/05b2befb-c601-421c-b8c6-aee1cdc6...,backward
209,data/stop/0024e456-a8d2-4bd0-872b-77145141039b...,stop
68,data/backward/012210d4-d9d4-423b-bb5a-d014c8e3...,backward
455,data/right/02b06b98-5783-4a70-a3ad-fbe3ac627b0...,right
377,data/left/00ac6dfd-4e4c-40ac-9a16-1f411b4bb7f6...,left
79,data/backward/0323baad-9da9-4e82-afe7-70ce0fb6...,backward
409,data/right/06a11bff-1ab9-4941-bf58-a6c1d9232f7...,right
278,data/stop/01b1b4b5-6473-42fb-a68d-193da2923185...,stop


In [11]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [12]:
#Create data generators
train_data_gen = ImageDataGenerator(
                               preprocessing_function=preprocess_input,
                               rotation_range=15,
                               width_shift_range=.2,
                               height_shift_range=.2,
                               horizontal_flip=True)
val_data_gen =  ImageDataGenerator(preprocessing_function=preprocess_input)

In [13]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    test_gen,
    x_col='Image',
    y_col = 'Label',
    target_size = (224,244),
    batch_size = 64,
    shuffle = False,
    class_mode='sparse',
    validate_filenames=True
)

Found 50 validated image filenames belonging to 5 classes.


In [14]:
train_generator = train_data_gen.flow_from_dataframe(
    train_gen,
    x_col='Image',
    y_col = 'Label',
    target_size = (224,244),
    batch_size = 64,
    shuffle = True,
    class_mode='sparse',
    validate_filenames=True
)
val_generator = val_data_gen.flow_from_dataframe(
    val_gen,
    x_col='Image',
    y_col = 'Label',
    target_size = (224,244),
    batch_size = 64,
    shuffle = True,
    class_mode='sparse',
    validate_filenames=True
)

Found 400 validated image filenames belonging to 5 classes.
Found 50 validated image filenames belonging to 5 classes.


ResNet

In [15]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2,l1


filepath = 'checkpoints/frozen.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 monitor='val_accuracy',
                                                save_best_only=True,
                                                 verbose=1,
                                                mode='max')
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=3,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
)
callbacks = [checkpoint,early_stop]

In [16]:
def create_model(drop_out=.1,neurons=256):
    resnet = ResNet50(include_top=False,weights='imagenet', pooling='avg')
    model = Sequential()
    model.add(resnet)
    model.add(Dropout(drop_out))
    model.add(Dense(5,activation='softmax'))
    return model

model = create_model(drop_out=.4)
model.compile(optimizer=Adam(learning_rate=0.001,decay=0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

2023-04-28 00:08:29.654269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 00:08:29.655286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 00:08:29.662584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 00:08:29.662702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [17]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=callbacks)

Epoch 1/50


2023-04-28 00:08:39.330773: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-28 00:08:39.939287: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-28 00:08:39.939681: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-28 00:08:39.939693: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-04-28 00:08:39.940091: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-28 00:08:39.940131: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-28 00:08:41.152081: I tensorflow/stream_executor/cuda/c

7/7 [==============================] - ETA: 0s - loss: 1.9787 - accuracy: 0.3050
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to checkpoints/frozen.hdf5
7/7 [==============================] - 25s 3s/step - loss: 1.9787 - accuracy: 0.3050 - val_loss: 4.8805 - val_accuracy: 0.2000
Epoch 2/50
7/7 [==============================] - ETA: 0s - loss: 1.1587 - accuracy: 0.5650
Epoch 2: val_accuracy improved from 0.20000 to 0.32000, saving model to checkpoints/frozen.hdf5
7/7 [==============================] - 19s 3s/step - loss: 1.1587 - accuracy: 0.5650 - val_loss: 2.7405 - val_accuracy: 0.3200
Epoch 3/50
7/7 [==============================] - ETA: 0s - loss: 0.9824 - accuracy: 0.6600
Epoch 3: val_accuracy did not improve from 0.32000
7/7 [==============================] - 16s 2s/step - loss: 0.9824 - accuracy: 0.6600 - val_loss: 2.8771 - val_accuracy: 0.2400
Epoch 4/50
7/7 [==============================] - ETA: 0s - loss: 0.6748 - accuracy: 0.7625
Epoch 4: val_accuracy 

In [18]:
from keras.models import load_model
model = load_model('checkpoints/frozen.hdf5')

In [19]:
for layer in model.layers:
  layer.trainable =  True

In [20]:
model.layers

In [21]:
filepath = 'checkpoints/finetuned2.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 monitor='val_accuracy',
                                                save_best_only=True,
                                                 verbose=1,
                                                mode='max')
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=3,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
)
callbacks = [checkpoint,early_stop]

In [22]:
model.compile(optimizer=Adam(learning_rate=0.0001,decay=0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
history2 = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=callbacks)

Epoch 1/50
7/7 [==============================] - ETA: 0s - loss: 0.0939 - accuracy: 0.9775
Epoch 1: val_accuracy improved from -inf to 0.64000, saving model to checkpoints/finetuned2.hdf5
7/7 [==============================] - 23s 3s/step - loss: 0.0939 - accuracy: 0.9775 - val_loss: 2.0278 - val_accuracy: 0.6400
Epoch 2/50
7/7 [==============================] - ETA: 0s - loss: 0.0547 - accuracy: 0.9850
Epoch 2: val_accuracy did not improve from 0.64000
7/7 [==============================] - 17s 2s/step - loss: 0.0547 - accuracy: 0.9850 - val_loss: 2.3860 - val_accuracy: 0.6400
Epoch 3/50
7/7 [==============================] - ETA: 0s - loss: 0.0513 - accuracy: 0.9875
Epoch 3: val_accuracy did not improve from 0.64000
7/7 [==============================] - 16s 2s/step - loss: 0.0513 - accuracy: 0.9875 - val_loss: 2.4074 - val_accuracy: 0.6400
Epoch 4/50
7/7 [==============================] - ETA: 0s - loss: 0.0500 - accuracy: 0.9875
Epoch 4: val_accuracy improved from 0.64000 to 0.680

In [24]:
model = load_model('checkpoints/finetuned2.hdf5')

In [25]:
pred = model.predict(test_generator)
pred = np.argmax(pred,axis=1)
print(classification_report(test_generator.classes,pred,target_names= label_list))

1/1 [==============================] - 2s 2s/step
              precision    recall  f1-score   support

    backward       1.00      0.57      0.73        14
     forward       0.35      0.86      0.50         7
        stop       1.00      0.50      0.67        14
        left       0.47      0.70      0.56        10
       right       0.67      0.40      0.50         5

    accuracy                           0.60        50
   macro avg       0.70      0.61      0.59        50
weighted avg       0.77      0.60      0.62        50

